**Date**: 2018-07-18

**Authors**: Yichen Fang

**Purpose**: Try more tests in order to confirm the shuffling result on 2018-07-17.

This notebook tests a naive alignment scheme that selects the first 1000 positions of a DNA sequence (add null positions to the end if a sequence is shorter than 1000). The sequences are shuffled before feeding into the neural network. The training data consists of 8712 sequences, and the validation data consists of 968 sequences.

**Background**:
- Based on the success of the shuffling result, this notebook runs more experiment to confirm that the result is indeed valid (not due to, say, random coincidence).

**Experiment**:

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

Load the `pickle` buffered list:

In [ ]:
with open("../data/list_buffer.txt", "rb") as buff:
    seq_record_list = pickle.load(buff)
len(seq_record_list)

In [ ]:
length_distribution = []
k = 0
for each_list in seq_record_list:
    length_distribution.append(len(each_list[3]))
    k += 1
    if k % 1000 == 1:
        print(k)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(length_distribution,bins=500)
plt.show()

The following cell randomly shuffles the sequences.

In [ ]:
from random import shuffle
shuffle(seq_record_list)

In [ ]:
import random
from random import shuffle

first_list = []
second_list = []
current = []

for i in range(len(seq_record_list)):
    current.append(seq_record_list.pop())
    if len(current) == 24:
        shuffle(current)
        random_select = random.randint(21, 24)
        first_list.extend(current[:random_select])
        second_list.extend(current[random_select:])
        current = []

shuffle(first_list)
shuffle(second_list)

seq_record_list = first_list + second_list

In [ ]:
len(second_list)

The following cell transforms the data into a format that is recognizable by the neural network model.

In [ ]:
# A helper function to flatten a 2d list to 1d.
# Input: [[1, 2], [2, 3], [3, 4, 5]]
# Output: [1, 2, 2, 3, 3, 4, 5]
def flatten(lst):
    new_lst = []
    for sub_lst in lst:
        for item in sub_lst:
            new_lst.append(item)
    return new_lst

# A helper function to transform a lst so that its length becomes read_len by:
# 1. If len(lst) > read_len, curtail the end of the lst.
# 2. If len(lst) < read_len, keep extending the end of the lst with 0 (NA).
def curtail(lst, read_len):
    if len(lst) > read_len:
        lst = lst[:read_len]
    else:
        for i in range(read_len - len(lst)):
            lst.append([0, 0, 0, 0, 0, 0, 0])
    return lst

# Produce the train-test split
# length_read: the length that you want all DNA sequences to conform to
def prepare_input(training_size, test_size, length_read):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    seq_count = 0
    while seq_count < training_size:
        X_train.append(flatten(curtail(seq_record_list[seq_count][3], length_read)))
        y_train.append(int(seq_record_list[seq_count][1]))
        seq_count += 1
    while seq_count < (training_size + test_size):
        X_test.append(flatten(curtail(seq_record_list[seq_count][3], length_read)))
        y_test.append(int(seq_record_list[seq_count][1]))
        seq_count += 1
    return X_train, y_train, X_test, y_test

# Turn list into numpy tensors that can directly feed into a neural network model
def to_np_array(X_train, y_train, X_test, y_test):
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    if len(y_train.shape) == 1:
        y_train = np.transpose(np.array([y_train]))
    X_test = np.array(X_test)
    y_test = np.transpose(np.array(y_test))
    if len(y_test.shape) == 1:
        y_test = np.transpose(np.array([y_test]))
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = prepare_input(10141, 611, 1000)
X_train, y_train, X_test, y_test = to_np_array(X_train, y_train, X_test, y_test)
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

We run the experiment with four LSTM layers, having 8, 8, 4, 4 units respectively, and 50 epoches:

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, CuDNNLSTM

In [ ]:
X_train_rnn = X_train.reshape(10141, 1000, 7)

In [ ]:
model = Sequential()
model.add(CuDNNLSTM(8, input_shape=(1000, 7), return_sequences=True))
model.add(CuDNNLSTM(8, return_sequences=True))
model.add(CuDNNLSTM(4, return_sequences=True))
model.add(CuDNNLSTM(4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_rnn, y_train, epochs=50, batch_size=128, validation_split=0.1)

**Result**:

The following cell **visualize** the training/validation accuracies and losses over each epoch.

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('epoches')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('epoches')
plt.legend()

plt.show()


**Conclusion**:

- The result is very promising. It confirms two things:
    1. The result on 2018-07-17 is not mere coincidence.
    2. As shown on the graph above, the trend of accuracy shows no sign of fading at epoch 50 and an accuracy of 85%, indicating that the limit has not been reached yet.
- My next steps are:
    1. Test the limit of the model.
    2. Speed up the model.